<a href="https://colab.research.google.com/github/Zilliax-Barry/FYP_Deep-Learning-and-WiFi-based-Gesture-Recognition/blob/main/ReviewSummarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [2]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from collections import OrderedDict
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import pandas as pd
import multiprocessing as mp
import datetime
#from helpers.duallogger import loggersetup
#from helpers.filehelper import is_not_empty_file_exists, write_to_file, load_from_file
import logging

from nltk.corpus import stopwords
import nltk
import re
import heapq
import collections
import operator
import numpy as np
from numpy import dot
from numpy.linalg import norm

Data Process

In [4]:
from pandas.io.json import json_normalize
import pandas as pd
import json

import json

count = 0
# 'reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',  'overall', 'summary', 'unixReviewTime', 'reviewTime'
reviewerID = []
asin = []
reviewerName = []
helpful = []
reviewText = []
overall = []
summary = []
unixReviewTime = []
reviewTime = []

with open('./Grocery_and_Gourmet_Food_5.json') as f:
  for line in f:
    # line type: 'string'
    # {"reviewerid": "a3aeguhvtc5yf4", "asin": "b000edm70y"...}    
    dicts= json.loads(line.strip())
    # print(dicts)
    count = count + 1
    if 'reviewerID' in dicts:
      reviewerID.append(dicts['reviewerID'])
    else:
      reviewerName.append('noID')  
    if 'reviewerName' in dicts:
      reviewerName.append(dicts['reviewerName'])
    else:
      reviewerName.append('noName')  
    if 'reviewText' in dicts:
      reviewText.append(dicts['reviewText'])
    else:
      reviewText.append('noName')  

print(count)  



151254


In [5]:
# Create the nlp object
nlp = spacy.load("en_core_web_sm")

print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [6]:
# pre-process the reviewerText
nltk.download('punkt')
nltk.download('stopwords')
stopwords_nltk = stopwords.words('english')
stopwords_spacy = list(STOP_WORDS)
stopwords_spacy.append('\n')
stopwords = stopwords_nltk + list(set(stopwords_spacy) - set(stopwords_nltk))

print("sw nltk: ", len(stopwords_nltk))
print("sw spacy: ", len(stopwords_spacy))
print("combined: ", len(stopwords))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


sw nltk:  179
sw spacy:  327
combined:  383


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
# pre-process the reviewerText
global prenlp
prenlp = spacy.load("en_core_web_sm", disable=['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner'])
prenlp.add_pipe('sentencizer')
punctuations = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~©.'
extractText = []

for i in range(len(reviewText)):
  text = reviewText[i].lower()
  review = prenlp(text)
  # POS tagging & tokenize
  sentence = nlp(review.text, disable=['parser', 'ner'])
  tokens = [tok.lemma_.lower().strip() for tok in sentence if tok.lemma_ != '-PRON-']
  tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
  tokens = ' '.join(tokens)
  extractText.append(tokens)



In [8]:
extractSentence = []
for i in range(len(reviewText)):
  doc = prenlp(reviewText[i]) 
  extractSentence.append([])
  for idx, sentence in enumerate(doc.sents):
      extractSentence[i].append(sentence.text)      

In [9]:
print(reviewText[300])
print(extractText[300])
print(extractSentence[300])

this ghost combo sauce hits different areas of the mouth than the red/orange/yellow/chocolate Habanerohighly recommended for a spice lovers aresenalfor best kick results combine with another multi-Habanero saucemade in costa rica
ghost combo sauce hit different area mouth red orange yellow chocolate habanerohighly recommend spice lover aresenalfor good kick result combine multi habanero saucemade costa rica
['this ghost combo sauce hits different areas of the mouth than the red/orange/yellow/chocolate Habanerohighly recommended for a spice lovers aresenalfor best kick results combine with another multi-Habanero saucemade in costa rica']


Review Summarizer

In [63]:
# Collection Frequency
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

word_token = {}
cleanText = []

for i in range(len(extractText)):
  #print(reviewText[i])
  eT_join = extractText[i]
  # eT_join the text in the ith reviewText  
  # words divide maynot only by '.' 
  eT_join = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', eT_join)
  cleanText.append(eT_join)
  #print(eT_join)
  token = re.findall(r"[\w']+", eT_join)
  #print(token)
  #
  text = prenlp(cleanText[i])
  for sent in text.sents:
    for tok in sent:
      #print('============')
      #print(tok)
      if str(tok) not in contractions:
        #print(tk)
        if str(tok) not in word_token:
          word_token[str(tok)] = 1
        else:
          word_token[str(tok)] += 1

print(word_token)



{'flavor': 84349, 'kit': 982, 'kat': 98, 'taste': 101541, 'unique': 1312, 'bit': 21376, 'different': 10738, 'thing': 15994, 'bothersome': 25, 'price': 22814, 'think': 28546, 'expensive': 5508, '    ': 20621, 'buy': 37595, 'impulse': 74, 'come': 18740, 'japan': 226, 'amuse': 28, 'family': 7182, 'weird': 1748, 'stamp': 119, 'marking': 17, 'package': 15335, 'fun': 1717, 'month': 4702, 'arrive': 3669, 'long': 12115, 'hope': 4136, 'interesting': 1829, 'flavorful': 3520, 'weak': 2882, 'standard': 2195, 'milk': 15764, 'chocolate': 30438, 'green': 9654, 'tea': 48934, 'subtle': 1952, 'sugar': 29066, 'sweet': 26122, 'wafer': 610, 'crispy': 1858, 'good': 89856, 'anemic': 24, 'us': 154, 'i': 24255, "'m": 1386, 'happy': 5597, 'need': 16511, 'great': 46895, 'gift': 2770, 'fan': 6109, 'purchase': 11803, 'candy': 8108, 'sea': 2084, 'curious': 497, 'like': 105397, 'smooth': 6203, 'order': 15732, 'plan': 2020, 'routine': 260, 'look': 16153, 'forward': 1515, 'try': 49155, 'hear': 1489, 'popular': 923, 'r

In [11]:
print(reviewText[400])
print(extractText[400])
print(cleanText[400])

warning buy eat fruit buy use fruit flavor vehicle tajin good good good amazing orange grape apple course watermelon
warning buy eat fruit buy use fruit flavor vehicle tajin good good good amazing orange grape apple course watermelon


In [12]:
# word_token
print("Numbers of tokens:", len(word_token))

Numbers of tokens: 1


In [13]:
def cos_sim(a, b):
  return dot(a, b)/(norm(a)*norm(b))

Choose one reviewText to summarize

In [117]:
#choose review index
from heapq import nlargest

index = 200

print('Review Text: '+ reviewText[index])
nouns = {}
adj = {}
sen_token = {}

# Make the spaCy doc obj for the sentence
renlp = spacy.load("en_core_web_sm")
text = renlp(cleanText[index])

for sent in text.sents:
  for tok in sent:
    print(tok, tok.pos_, tok.lemma_)
    if str(tok.lemma_) not in sen_token:
      sen_token[str(tok.lemma_)] = 1
    else:
      sen_token[str(tok.lemma_)] += 1

    if str(tok.pos_) == 'NOUN':
      if str(tok.lemma_) not in nouns:
        nouns[str(tok.lemma_)] = 1
      else:
        nouns[str(tok.lemma_)] += 1
        
    elif str(tok.pos_) == 'ADJ':
      if str(tok.lemma_) not in adj:
        adj[str(tok.lemma_)] = 1
      else:
        adj[str(tok.lemma_)] += 1

print(sen_token)
print(nouns)
print(adj)

Review Text: fragrant, evocative, delicious and very lovely.  These are not words usually used to describe tea.  I am sure this tea is loaded with who knows what flavorings & spices etc but in a steaming cup, this is heaven.  with some milk,  heaven plus
fragrant PROPN fragrant
evocative ADJ evocative
delicious ADJ delicious
lovely ADJ lovely
word NOUN word
usually ADV usually
use VERB use
describe NOUN describe
tea NOUN tea
sure ADJ sure
tea NOUN tea
load NOUN load
know VERB know
flavoring VERB flavor
spice NOUN spice
etc X etc
steaming NOUN steaming
cup NOUN cup
heaven PROPN heaven
milk NOUN milk
heaven PROPN heaven
plus CCONJ plus
{'fragrant': 1, 'evocative': 1, 'delicious': 1, 'lovely': 1, 'word': 1, 'usually': 1, 'use': 1, 'describe': 1, 'tea': 2, 'sure': 1, 'load': 1, 'know': 1, 'flavor': 1, 'spice': 1, 'etc': 1, 'steaming': 1, 'cup': 1, 'heaven': 2, 'milk': 1, 'plus': 1}
{'word': 1, 'describe': 1, 'tea': 2, 'load': 1, 'spice': 1, 'steaming': 1, 'cup': 1, 'milk': 1}
{'evocative':

In [118]:
print('=============')
# most frqe noun ,adj sentence
max_noun = nlargest(1, nouns, key = nouns.get)
max_noun = str(max_noun[0])

max_adj = nlargest(1, adj, key = adj.get)
max_adj = str(max_adj[0])

print('max_noun', max_noun)
print('max_adj', max_adj)
print(cleanText[index])

# \s{1,} to ignore the other .'s
sentences = re.split("[.!?]\s{1,}", reviewText[index])
print(sentences)
# a quick clean for each sentences
extractSentence = []
cleanSentence = []
for s in sentences:
  sentence = renlp(s)
  print(sentence)
  # POS tagging & tokenize
  d = nlp(sentence.text, disable=['parser', 'ner'])
  k = [tok.lemma_.lower().strip() for tok in d if tok.lemma_ != '-PRON-']
  k = [tok for tok in k if tok not in stopwords and tok not in punctuations]
  k = ' '.join(k)
  print(k)
  extractSentence.append(k)

print(len(extractSentence))
print('=====================')
for i in range(len(extractSentence)):
  print('===')
  print(extractSentence[i])
  eS_join = extractSentence[i]
  # eS_join the sentence in the ith extractSentence  
  # words divide maynot only by '.' 
  eS_join = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', eS_join)
  print(eS_join)
  cleanSentence.append(eS_join)

print('=====================')
# print(extractSentence)
print(cleanSentence)


max_noun tea
max_adj evocative
fragrant evocative delicious lovely word usually use describe tea sure tea load know flavoring spice etc steaming cup heaven milk heaven plus
['fragrant, evocative, delicious and very lovely', 'These are not words usually used to describe tea', 'I am sure this tea is loaded with who knows what flavorings & spices etc but in a steaming cup, this is heaven', 'with some milk,  heaven plus']
fragrant, evocative, delicious and very lovely
fragrant evocative delicious lovely
These are not words usually used to describe tea
word usually use describe tea
I am sure this tea is loaded with who knows what flavorings & spices etc but in a steaming cup, this is heaven
sure tea load know flavoring spice etc steaming cup heaven
with some milk,  heaven plus
milk heaven plus
4
===
fragrant evocative delicious lovely
fragrant evocative delicious lovely
===
word usually use describe tea
word usually use describe tea
===
sure tea load know flavoring spice etc steaming cup he

In [119]:
targetNounIndex = []
targetADJIndex = []
print(max_noun)

for i in range(len(cleanSentence)):
  s = cleanSentence[i]
  s = str(s)
  print(s)

  if max_noun in s:
    targetNounIndex.append(i)
  if max_adj in s:
    targetADJIndex.append(i)
  
print(targetNounIndex)
print(targetADJIndex)

tea
fragrant evocative delicious lovely
word usually use describe tea
sure tea load know flavoring spice etc steaming cup heaven
milk heaven plus
[1, 2]
[0]


In [122]:

# tf-idf for a sentence with the most freq noun
targetNounSentences = []
targetADJSentences = []
nounVector = [] # this is the vector for one sentence
adjVector = []
targetNounVector = [] # this is the vector list for all target sentences
targetADJVector = []

for i in targetNounIndex:
  targetNounSentences.append(cleanSentence[i])
for i in targetADJIndex:
  targetADJSentences.append(cleanSentence[i])
print(targetNounSentences)
print(targetADJSentences)

# word frequence here is sen_token, for all is word_token
for i in range(len(targetNounIndex)):
  nounVector = []
  targetS = nlp(targetNounSentences[i])
  #print(tS)
  for word in word_token:
    #print(word)
    tS = str(targetS)
    if word in tS:
      #print(word)
      wd= str(word)
      tfidf_value = (1 + np.log(word_token[wd])) * np.log(len(reviewText)/word_token[wd])
    else:
      tfidf_value = 0
    nounVector.append(tfidf_value)

  print(len(nounVector))  
  targetNounSentences.append(nounVector)
  print(targetNounSentences[i])
# for end

# word frequence here is sen_token, for all is word_token
for i in range(len(targetADJIndex)):
  adjVector = []
  targetS = nlp(targetADJSentences[i])
  #print(tS)
  for word in word_token:
    #print(word)
    tS = str(targetS)
    if word in tS:
      #print(word)
      wd= str(word)
      tfidf_value = (1 + np.log(word_token[wd])) * np.log(len(reviewText)/word_token[wd])
    else:
      tfidf_value = 0
    adjVector.append(tfidf_value)

  print(len(adjVector))  
  targetADJSentences.append(adjVector)
  print(targetADJSentences[i])
# for end

['word usually use describe tea', 'sure tea load know flavoring spice etc steaming cup heaven']
['fragrant evocative delicious lovely']
70562
word usually use describe tea
70562
sure tea load know flavoring spice etc steaming cup heaven
70562
fragrant evocative delicious lovely


In [128]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
# determine the best sentence
min_noun_dis = 10000
min_noun_index = 0
min_adj_dis = 10000
min_adj_index = 0
# noun
for i in range(len(targetNounVector)):
  targetNounMatrix = np.column_stack(targetNounVector[i])
  K_estimate = KMeans(n_clusters=1, random_state=0).fit(targetNounMatrix)
  cluster_ids = K_estimate.labels_
  centroids = K_estimate.cluster_centers_
  # eu-distance
  dist = euclidean_distances(targetNounMatrix, centroids[cluster_ids])
  if dist < min_noun_dis:
    min_noun_dis = dist
    min_noun_index = i

print('Best noun index:', min_noun_index)

# adj
for i in range(len(targetADJVector)):
  targetADJMatrix = np.column_stack(targetADJVector[i])
  K_estimate = KMeans(n_clusters=1, random_state=0).fit(targetADJMatrix)
  cluster_ids = K_estimate.labels_
  centroids = K_estimate.cluster_centers_
  # eu-distance
  dist = euclidean_distances(targetADJMatrix, centroids[cluster_ids])
  if dist < min_adj_dis:
    min_adj_dis = dist
    min_adj_index = i

print('Best ADJ index:', min_adj_index)

result = 'The sentence is too short to get a summary.' # thats why the min_index = 0

if targetNounIndex[min_noun_index] == targetADJIndex[min_adj_index]:
  result = cleanSentence[min_noun_index]
else:
  result = targetNounSentences[min_noun_index] + '. '+ targetADJSentences[min_adj_index]+ '. '

print('Summarizer Result is: ' +result)


Best noun index: 0
Best ADJ index: 0
Summarizer Result is: word usually use describe tea. fragrant evocative delicious lovely. 
